Baños no compartidos

Creamos una clase auxiliar

In [ ]:
%%writefile Bath.java
import java.util.concurrent.Semaphore;

public class Bath implements Runnable
{
    /* Constantes */
    private static final int TIME_IN_BATH = 3000;
    private static final int TIME_WAITING = 3000;
    private static final int BATH_CAPACITY = 2;
    private static final String MAN = "H";
    private static final String WOMAN = "M";



    private String gender;
    private int number;


    public static int women_counter = 0;
    public static int men_counter = 0;
    public static Semaphore sem_access_bath = new Semaphore(BATH_CAPACITY);

    public Bath(String gender, int number)
    {
        this.gender = gender;
        this.number = number;
    }

    public void use_bath() throws InterruptedException
    {
        /* Si un hombre quiere entrar y no hay mujeres en el baño, entonces pasa */
        if (this.gender.equals(MAN) && get_women_counter() == 0)
        {
            /* Si es el primer hombre, entonces bloqueo el acceso a mujeres */
            Main.enter_bath(number);

            /* Dibujo un hombre en pantalla */
            Bath.increment_men_counter();

            /* Simula el tiempo que el hilo esta usando el baño */
            Thread.sleep(TIME_IN_BATH);

            Main.leave_bath(number);
            /* Borro al hombre en pantalla */
            Bath.decrement_men_counter();


        }
        /* Entra mujer y contador de hombres debe estar en 0 */
        else if (this.gender.equals(WOMAN) && get_men_counter() == 0)
        {
            Main.enter_bath(number);

            /* Dibujo una mujer en pantalla */
            Bath.increment_women_counter();

            /* Simula el tiempo que el hilo esta usando el baño */
            Thread.sleep(TIME_IN_BATH);

            /* Se va y actualiza la pantalla */
            Main.leave_bath(number);
            Bath.decrement_women_counter();
        } else
        {
            /* Si no pudo ingresar sea hombre o mujer, se va un rato y vuelve a intentar (pero con su lugar reservado para evitar inanicion) */
            Thread.sleep(TIME_WAITING);
            use_bath();
        }
    }

    public void try_access_bath(){
        Main.waiting(gender, number);
        P("access_bath");
        try
        {
            use_bath();
        } catch (Exception e)
        {
            e.printStackTrace();
        }
        V("access_bath");
    }


    @Override
    public void run()
    {
        try_access_bath();
    }

    public void P(String name)
    {
        try
        {
            if (name.equals("access_bath"))
            {
                sem_access_bath.acquire();
            }

        } catch(InterruptedException e)
        {
            e.printStackTrace();
        }

    }

    public void V(String name)
    {
        if (name.equals("access_bath"))
        {
            sem_access_bath.release();
        }
    }

    public static synchronized void increment_men_counter()
    {
        men_counter++;
        Main.update_screen();
    }

    public static synchronized void decrement_men_counter()
    {
        men_counter--;
        Main.update_screen();
    }
    public static synchronized int get_men_counter()
    {
        return Bath.men_counter;
    }


    public static synchronized void increment_women_counter()
    {
        women_counter++;
        Main.update_screen();
    }

    public static synchronized void decrement_women_counter()
    {
        women_counter--;
        Main.update_screen();
    }

    public static synchronized int get_women_counter()
    {
        return Bath.women_counter;
    }

}

Overwriting Bath.java


Creamos nuestro main en Java

In [ ]:
%%writefile Main.java
import java.util.Scanner;


public class Main
{

    private static final String MAN = "H";
    private static final String WOMAN = "M";


    public static int access_counter = 1;
    public static void main(String[] args)
    {
        System.out.println("Este programa simula el acceso a un baño donde solo hombres entran o solo mujeres.");
        System.out.println("Ingrese 'H' para simular el ingreso de un hombre");
        System.out.println("Ingrese 'M' para simular el ingreso de una mujer\nPara finalizar, ingrese 'salir'");

        Scanner scanner = new Scanner(System.in);
        int i = 1;

        while (true)
        {
            System.out.println("'H' , 'M' o 'salir':");
            String input = scanner.nextLine();

            if (input.equals(MAN))
            {
                Thread man_access = new Thread(new Bath(MAN, i));
                man_access.start();
                i++;
            }
            else if (input.equals(WOMAN))
            {
                Thread woman_access = new Thread(new Bath(WOMAN, i));
                woman_access.start();
                i++;
            }
            else if (input.equals("salir"))
            {
                break;
            }
            else
            {
                System.out.println("Entrada no valida.");
            }
        }

        scanner.close();
    }


    public static synchronized void update_screen()
    {
        int counter = 0;
        String gender = "";
        String out = "Baño: ";
        System.out.println();
        System.out.println("=====================" + Main.access_counter + "=====================");

        int man_counter = Bath.get_men_counter();
        int woman_counter = Bath.get_women_counter();

        counter = man_counter != 0 ? man_counter : woman_counter;
        gender = man_counter !=0 ? MAN : WOMAN;

        for(int i = 0; i < counter; i++)
        {
            out += gender + " ";
        }
        System.out.println(out);
        System.out.println("=====================");
        Main.access_counter++;
    }

    public static synchronized void enter_bath(int number)
    {
        System.out.println("Persona: " + number + " ingresando");
    }

    public static synchronized void leave_bath(int number)
    {
        System.out.println("Persona: " + number + " saliendo");
    }

    public static synchronized void waiting(String gender, int number)
    {
        System.out.println("Persona: " + number + " de genero: " + gender + " esta esperando");
    }
}

Overwriting Main.java


Compilamos nuestros .java

In [ ]:
!javac Bath.java
!javac Main.java

Ejecutamos nuestro ejercicio en Baños no compartidos

In [ ]:
!java Main

Este programa simula el acceso a un baño donde solo hombres entran o solo mujeres.
Ingrese 'H' para simular el ingreso de un hombre
Ingrese 'M' para simular el ingreso de una mujer
Para finalizar, ingrese 'salir'
'H' , 'M' o 'salir':
H
'H' , 'M' o 'salir':
Persona: 1 de genero: H esta esperando
Persona: 1 ingresando

=====================1=====================
Baño: H 
M
'H' , 'M' o 'salir':
Persona: 2 de genero: M esta esperando
Persona: 1 saliendo

=====================2=====================
Baño: 
M
'H' , 'M' o 'salir':
Persona: 3 de genero: M esta esperando
Persona: 3 ingresando

=====================3=====================
Baño: M 
Persona: 2 ingresando

=====================4=====================
Baño: M M 
Persona: 3 saliendo

=====================5=====================
Baño: M 
Persona: 2 saliendo

=====================6=====================
Baño: 
salir


Babuinos

Creamos el archivo con la logica en Python

In [ ]:
%%writefile babuinos.py
import sys
import threading
import time
import random


MAX_BABUINOS = 5


def babuinos(N, M):
    semaforo = threading.Semaphore(MAX_BABUINOS)
    sem_izq = threading.Semaphore(MAX_BABUINOS)
    sem_der = threading.Semaphore(MAX_BABUINOS)
    babuinos = []

    for _ in range(N):
        babuinos.append(threading.Thread(target=cruzar_soga, args=(
            semaforo, sem_izq, sem_der, True)))

    for _ in range(M):
        babuinos.append(threading.Thread(target=cruzar_soga, args=(
            semaforo, sem_izq, sem_der, False)))

    random.shuffle(babuinos)
    for babuino in babuinos:
        babuino.start()
        time.sleep(random.randint(1, 2))


def cruzar_soga(semaforo, sem_izq, sem_der,  es_izquierda):
    semaforo.acquire()
    if es_izquierda and sem_der._value == MAX_BABUINOS:
        sem_izq.acquire()
        cruzar("Cruzando hacia la izquierda")
        sem_izq.release()
    elif sem_izq._value == MAX_BABUINOS:
        sem_der.acquire()
        cruzar("Cruzando hacia la derecha")
        sem_der.release()

    semaforo.release()


def cruzar(st):
    print(st)
    time.sleep(random.randint(1, 2))


def main():
    if len(sys.argv) != 3:
        print("""Por favor provea dos argumentos:
                N (babuinos que cruzan de izquierda a derecha)
                M (babuinos que cruzan de derecha a izquierda)""")
        return

    N = int(sys.argv[1])
    M = int(sys.argv[2])

    babuinos(N, M)


if __name__ == "__main__":
    main()

Overwriting babuinos.py


Damos ejecucion al ejercicio de Babuinos

In [ ]:
!python babuinos.py 15 43

Cruzando hacia la izquierda
Cruzando hacia la izquierda
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la derecha
Cruzando hacia la izquierda
Cruzando hacia la derecha
Cruzando hacia la izquierda
Cruzando hacia la derecha
Cruzando hacia la derecha
Traceback (most recent call last):
  File "/content/babuinos.py", line 63, in <module>
    main()
  File "/content/babuinos.py", line 59, in main
    babuinos(N, M)
  File "/content/babuinos.py", line 27, in babuinos
    time.sleep(random.randint(1, 2))
KeyboardInterrupt
Exception ignored in: <module 'threading' from '/usr/lib/python3.10/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1567, in _shutdown
    lock.acquire()
KeyboardInterrupt: 
^C


Supermercado

Creamos el archivo de Supermercado en C++

In [ ]:
%%writefile main.cpp
#include <fcntl.h>
#include <semaphore.h>
#include <unistd.h>

#include <cstdlib>
#include <iostream>
#include <list>
#include <mutex>
#include <thread>
#include <vector>

using namespace std;

#define ERROR_NPARAMETERS -1
#define ERROR_NCLIENTS -2

#define NADMITTED_PARAMTERS 2
#define NRESTOKERS 2
#define MAX_BUY 2
#define MIN_BUY 1

int nClients = 0;
vector<sem_t *> semaphores;
sem_t *reponerS;
sem_t *waitRestock;
mutex gondolaAccesMutex;
mutex waitingArea;
int productCount = 10;
bool noMoreClients = false;

int parametersValidation(int nParamters, char const *paramters[]);
list<thread> createRestockers(int nRestockers);
void reponer(int numb_restocker, sem_t *myInit, sem_t *nextRepo);
list<thread> createClients(int nClients);
void comprar(int order);

int main(int argc, char const *argv[])
{
  int validation = parametersValidation(argc, argv);

  if (validation < 0)
  {
    cout << "Error - " << validation << endl;
    return validation;
  }

  cout << "N clients - " << nClients << endl;

  reponerS = sem_open("reponer", O_CREAT, 0600, 0);
  waitRestock = sem_open("esperar", O_CREAT, 0600, 0);
  waitingArea.lock();
  list<thread> listRestockers = createRestockers((NRESTOKERS));
  list<thread> listClients = createClients(nClients);

  waitingArea.unlock();
  for (auto &client : listClients)
  {
    client.join();
  }
  cout << "Close SuperMarket" << endl;
  noMoreClients = true;
  for (auto &restocker : listRestockers)
  {
    restocker.join();
  }
  int indxNameSemaphore = 0;
  string nameSemaphoreClose;
  for (auto &semaphore : semaphores)
  {
    nameSemaphoreClose = "Número: " + std::to_string(indxNameSemaphore);
    sem_close(semaphore);
    sem_unlink(nameSemaphoreClose.c_str());
    indxNameSemaphore++;
  }
  sem_close(reponerS);
  sem_close(waitRestock);
  sem_unlink("reponer");
  sem_unlink("esperar");
  return 0;
}

int parametersValidation(int nParamters, char const *paramters[])
{
  if (nParamters != NADMITTED_PARAMTERS)
  {
    return ERROR_NPARAMETERS;
  }
  else
  {
    nClients = atoi(paramters[1]);
    if (nClients < 0)
    {
      return ERROR_NCLIENTS;
    }
  }
  return 0;
}

list<thread> createRestockers(int nRestockers)
{
  list<thread> restockers;
  string nameSemaphore;
  for (int i = 0; i < nRestockers; i++)
  {
    nameSemaphore = "Número: " + std::to_string(i);
    semaphores.push_back(sem_open(nameSemaphore.c_str(), O_CREAT, 0600, i == 0 ? 1 : 0));
  }
  for (int i = 0; i < nRestockers; i++)
  {
    restockers.push_back(std::thread(reponer, i + 1, semaphores.at(i), semaphores.at(i + 1 < nRestockers ? i + 1 : 0)));
  }

  return restockers;
}

void reponer(int numb_restocker, sem_t *myInit, sem_t *nextRepo)
{
  while (1)
  {
    while (sem_trywait(myInit) != 0)
    {
      sleep(5);
      if (noMoreClients == true) return;
    }
    while (sem_trywait(reponerS) != 0)
    {
      sleep(1);
      if (noMoreClients == true) return;
    }
    cout << "Restocker N°" << numb_restocker << " restocking" << endl;
    gondolaAccesMutex.lock();
    productCount = 10;
    gondolaAccesMutex.unlock();
    sem_post(nextRepo);
    sem_post(waitRestock);
  }
}

list<thread> createClients(int nClients)
{
  list<thread> clients;
  int nRand;
  for (int i = 0; i < nClients; i++)
  {
    nRand = MIN_BUY + (std::rand() % (MAX_BUY - MIN_BUY + 1));
    clients.push_back(std::thread(comprar, nRand));
  }

  return clients;
}

void comprar(int order)
{
  waitingArea.lock();
  gondolaAccesMutex.lock();
  cout << " I want " << order << " products..." << endl;

  if (productCount - order < 0)
  {
    cout << "   - I'll wait for " << order << " more." << endl;
    gondolaAccesMutex.unlock();
    sem_post(reponerS);
    sem_wait(waitRestock);
    gondolaAccesMutex.lock();
  }
  productCount -= order;
  cout << " Thanks :D " << endl;
  gondolaAccesMutex.unlock();
  waitingArea.unlock();
}

Compilamos nuestro main.cpp

In [ ]:
!g++ -o SuperMarket main.cpp

Ejecutamos el ejercicio pasando como parametro la cantidad de clientes que compraran

In [ ]:
!./SuperMarket 30